In [ ]:
# Python packages and utilities
import importlib
import logging
import os
import pickle
from datetime import datetime

import numpy as np
import pandas as pd
from IPython.display import HTML, display
from pickle import dump

# matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager

# Update the matplotlib font configuration
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['DejaVu Sans']

# mlxtend
import mlxtend
from mlxtend.plotting import plot_pca_correlation_graph

# RDKit
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Crippen, Descriptors, Draw, rdMolDescriptors
from rdkit.Chem.Draw import DrawingOptions, IPythonConsole
from rdkit.ML.Cluster import Butina

# scikit-learn
import sklearn
from sklearn import linear_model, metrics, svm
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor
from sklearn.feature_selection import RFE, RFECV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel as C, DotProduct, Matern, RBF, WhiteKernel
from sklearn.linear_model import LassoCV, LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, KFold, cross_validate, train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.tree import DecisionTreeRegressor

# scikitplot library
import scikitplot as skplt

# seaborn
try:
    import seaborn as sns
except ModuleNotFoundError:
    !pip install seaborn
    import seaborn as sns

# own module
from pythia import classification_metrics as cmetrics
from pythia import fingerprints_generation as fp
from pythia import molecules_and_structures as ms
from pythia import plots as pltsk
from pythia import scaling
from pythia import workflow_functions as wf

# utility
%load_ext autoreload
%autoreload 2
%aimport

# Logging setup
logging.basicConfig(format='%(message)s')
log = logging.getLogger()
log.setLevel(logging.INFO)

# Random seed setup
random_seed = 10459
np.random.seed = random_seed
np.random.RandomState(random_seed)
log.info(f"Random seed fixed as {random_seed}")


This notebook apart from reproducing the results of the paper also attempts to show you how to use precalculated descriptors to train your model. In this case we used DFT descriptors, however you could use any type of descriptors you wish. It explains how to use PCA (although it is not used for the results of this paper) as well as how to get the most important features from LASSOCV. This is used in the results of the paper. Please follow step by step of feel free to skip steps you might not be interested in like PCA or LASSOCV interpretation.

# Load target data

First we read the dataset from a csv file. A csv file is a comma separated file, where each row is a data point.
pd.read_csv() is used to read the input csv file that contain the dataset. The dataset is stored in a dataframe, which is a data structure provided by the pandas library. A dataframe is a two-dimensional data structure, i.e. data is aligned in a tabular fashion in rows and columns. The dataframe can be indexed by column names.
Here, we also reformat the column names to replace spaces with underscores and make the column names all lower case. This is done to make it easier to work with the data.

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 10)

We have one csv with the DFT data and one with the targets (ddg)

In [ ]:
data = pd.read_csv("DFTdata.csv",header=None)

In [ ]:
ddg = pd.read_csv("DDG.csv",header=None)

In [ ]:
ddg = ddg[0]

Here I make for you a folder to save all the models you create with this notebook

In [ ]:
os.makedirs("results_reg_DFT", exist_ok=True)
os.chdir("results_reg_DFT")

# PCA Example
Here we show how you can perform PCA to reduce the features.

If you do not know what PCA is or how it works please read
https://en.wikipedia.org/wiki/Principal_component_analysis 

In [ ]:
X = data
y = ddg # is not used

scaler = MinMaxScaler()
data_rescaled = scaler.fit_transform(X)

In [ ]:
pca = PCA().fit(data_rescaled)
plt.rcParams["figure.figsize"] = (20,6)
fig, ax = plt.subplots()
xi = np.arange(0, 44, step=1) # change 44 according to number of features!!!
y = np.cumsum(pca.explained_variance_ratio_)

plt.ylim(0.0,1.1)
plt.plot(xi, y, marker='o', linestyle='--', color='b')

plt.xlabel('Number of Components')
plt.xticks(np.arange(0, 44, step=1)) #change here as well
plt.ylabel('Cumulative variance (%)')
plt.title('The number of components needed to explain variance')

plt.axhline(y=0.95, color='r', linestyle='-')
plt.text(0.5, 0.85, '95% cut-off threshold', color = 'red', fontsize=16)

ax.grid(axis='x')
plt.show()

The important descriptors are 12 according to the plot above

In [ ]:
model = PCA(n_components = 0.95, whiten=True).fit(data_rescaled)
X_pc = model.transform(data_rescaled)

n_pcs= model.components_.shape[0]
print(n_pcs)

# get the index of the most important feature on EACH component i.e. largest absolute value
# using LIST COMPREHENSION HERE
most_important = [np.abs(model.components_[i]).argmax() for i in range(n_pcs)]

# if your features have names, as they should, you can do the following. if not no need to remove from comments
# Name all features in order as they appear in your csv eg.
# initial_feature_names = ['HOMOsub', 'LUMOsub', 'C1', 'N1', 'Cindole', 'Dipolesub', 'L(N-C1)',
#        'B1(N-C1)', 'B5(N-C1)', 'L(N-C3)', 'B1(N-C3)', 'B5(N-C3)', 'L(N-C12)',
#        'B5(H-Cind+A2ole)', 'B5(N-C12)', 'L(H-Cindole)', 'B1(H-Cindole)',
#        'B5(H-Cindole)', 'HOMO', 'LUMO', 'TOTAL NH', 'TOTAL CN', 'TOTAL N49-H',
#        'Cl- Charge', 'TOTAL Hcharge', 'AVG H CHRG', 'DIPOLE', '49Cl NMR']
# get the names
# most_important_names = [initial_feature_names[most_important[i]] for i in range(n_pcs)]
# print(most_important_names)

# using LIST COMPREHENSION HERE AGAIN
# dic = {'PC{}'.format(i+1): most_important_names[i] for i in range(n_pcs)}

# If no names are included continue here
dic = {'PC{}'.format(i+1): most_important[i] for i in range(n_pcs)}
# build the dataframe
df = pd.DataFrame(dic.items())
df


make dataframe only containing the PCA features

In [ ]:
col_names = [col for col in data.columns if col in df[1].values and data[col].all()]

In [ ]:
col_names

In [ ]:
data_pca = data[col_names]
print(data_pca)

Now we call our classifiers 

In [ ]:
# Feel free to add or remove regressors you might want (or don't) to explore, here we just offer some examples.

kfold_reg_names = ["LassoCV","KNeighborsRegressor", "DecisionTreeRegressor", "SVR",
                   "BayesianRegr", "GaussianProcessRegressor", "RandomForestRegressor"]
kfold_regressors = [
    LassoCV(random_state=random_seed, cv=10,selection='random',max_iter=1000000),
    KNeighborsRegressor(),
    DecisionTreeRegressor(random_state=random_seed),
    svm.SVR(), #SVR takes some time be patient if you are not, comment it out
    linear_model.BayesianRidge(n_iter=100000),
    GaussianProcessRegressor(),
    RandomForestRegressor(random_state=random_seed)]# takes some time, you can remove some options for the parameters

kfold_regressors_parameters = {
    "LassoCV":{},
    "KNeighborsRegressor": {"n_neighbors": [ent for ent in range(2, 10, 1)]},
    "DecisionTreeRegressor": {"max_depth": [2, 3, 4, 5, 7, 10]},
    "SVR": {"kernel":['linear', 'poly', 'rbf'], "degree":[2,3], "gamma":['auto','scale'], "coef0":[0,1], 'C':[100]},    
    "BayesianRegr":{'alpha_1':[1e-06, 10], 'alpha_2': [1e-06,10],'lambda_1':[1e-06,10], 'lambda_2': [1e-06,10]},
    "GaussianProcessRegressor": {},  
    "RandomForestRegressor":{'n_estimators': [10,20],'criterion': ['friedman_mse', 'absolute_error', 'squared_error', 'poisson'],'max_depth': [3, 4, 5, 7, 10],
    'max_features': ['auto','sqrt','log2'],'bootstrap': [True, False],'warm_start': [True, False]}
}

And finally we train the model based on the features that PCA selected!

In [ ]:
%%capture
wf.kfold_test_regressor_with_optimization(data_pca,ddg, kfold_regressors, kfold_regressors_parameters, scale=False, cv=5, n_repeats=10, rgs_names=kfold_reg_names)

In [ ]:
directory_names = wf.directory_names(kfold_reg_names)

In [ ]:
data = wf.build_data_from_directory_regr(directory_names[0], max_folds=10)

In [ ]:
wf.metrics_for_regression(directories=directory_names)

Before you continue please take a moment to look at your results! If you continue blindly, some of the results might be overwritten by the following cells

# LASSOCV Example

Here I show you how to extract important descriptors from LASSO. LASSO uses the L1 regularization term that shrinks features to 0. If you dont know about LASSO read here https://en.wikipedia.org/wiki/Lasso_(statistics)

We need to import again our data (DFT descriptors and DDG target values) as we have made changes on the firstly imported data. Remember we have changed directories!

In [ ]:
data = pd.read_csv("../DFTdata.csv",header=None)
ddg = pd.read_csv("../DDG.csv",header=None)
ddg = ddg[0]

In [ ]:
Coefficients,Feats = [],[]

X = pd.read_csv("../DFTdata.csv",header=None)
Y = pd.read_csv("../DDG.csv",header=None)
Y=Y[0]

X = scaling.minmaxscale(X)

model=LassoCV(cv=10,selection='random',fit_intercept=True, max_iter=100000000).fit(X,Y)

#Training set
predicted=model.predict(X)

print('Root Mean Squared Error Training Set:', np.sqrt(metrics.mean_squared_error(Y, predicted)))
print('R2 Training Set:',r2_score(Y, predicted))


#Coefficients
log.info("\n-----\nCoefficients\n-----\n")
log.info(model.coef_)
print(len(model.coef_))
for i in range (len(model.coef_)):
    Coefficients.append(model.coef_[i])
    log.info(model.coef_[i])
Feat1=np.nonzero(model.coef_)
Feats.append(Feat1)
log.info(Feat1)
# X2=X[:,Feat1]
# X2.shape
print(model.intercept_)

# Now first we split 90%-10% (train-test)

We need to evaluate the performace of our model on an external test set. We know which data points we are using for test set, for consistency reasons. However if you have much more data and you dont know which ones you want to use as a training set, or you don't want to write them manually you could do something like this:

import pandas as pd

from sklearn.model_selection import train_test_split

df = pd.DataFrame(data)

X = df.drop('target', axis=1) y = df['target']

test_size = 0.2

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

print("X_test:") print(X_test)

print("y_test:") print(y_test)

In [ ]:
data = pd.read_csv("../DFTdata.csv", header = None)

In [ ]:
test_reactions = [21, 81, 11, 104, 44, 106, 54, 95, 115, 25, 57, 52]

In [ ]:
train_data = data.drop(labels =test_reactions, axis=0,inplace = False)

In [ ]:
test_data = pd.DataFrame()
test_data = data.iloc[test_reactions]

In [ ]:
ddg = pd.read_csv("../DDG.csv", header = None)
ddg = ddg[0]

In [ ]:
ddg_train = ddg.drop(labels =test_reactions, axis=0,inplace = False)

In [ ]:
ddg_test = ddg.iloc[test_reactions]

# Define Xtrain, Xtest, Ytrain, Ytest


In [ ]:
Xtrain = train_data
Xtest = test_data
Ytrain = ddg_train
Ytest = ddg_test

# Time for ML


In [ ]:
# Feel free to add or remove regressors you might want (or don't) to explore, here we just offer some examples.

kfold_reg_names = ["LassoCV","KNeighborsRegressor", "DecisionTreeRegressor", "SVR",
                   "BayesianRegr", "GaussianProcessRegressor", "RandomForestRegressor"]
kfold_regressors = [
    LassoCV(random_state=random_seed, cv=10,selection='random',max_iter=1000000),
    KNeighborsRegressor(),
    DecisionTreeRegressor(random_state=random_seed),
    svm.SVR(), #SVR takes some time be patient if you are not, comment it out
    linear_model.BayesianRidge(n_iter=100000),
    GaussianProcessRegressor(),
    RandomForestRegressor(random_state=random_seed)]# takes some time, you can remove some options for the parameters

kfold_regressors_parameters = {
    "LassoCV":{},
    "KNeighborsRegressor": {"n_neighbors": [ent for ent in range(2, 10, 1)]},
    "DecisionTreeRegressor": {"max_depth": [2, 3, 4, 5, 7, 10]},
    "SVR": {"kernel":['linear', 'poly', 'rbf'], "degree":[2,3], "gamma":['auto','scale'], "coef0":[0,1], 'C':[100]},    
    "BayesianRegr":{'alpha_1':[1e-06, 10], 'alpha_2': [1e-06,10],'lambda_1':[1e-06,10], 'lambda_2': [1e-06,10]},
    "GaussianProcessRegressor": {},  
    "RandomForestRegressor":{'n_estimators': [10,20],'criterion': ['friedman_mse', 'absolute_error', 'squared_error', 'poisson'],'max_depth': [3, 4, 5, 7, 10],
    'max_features': ['auto','sqrt','log2'],'bootstrap': [True, False],'warm_start': [True, False]}
}

In [ ]:
%%capture
wf.split_test_regressors_with_optimization(Xtrain,Ytrain,Xtest,Ytest, kfold_regressors, kfold_regressors_parameters, scale=True, cv=5, n_repeats=10, rgs_names=kfold_reg_names)